In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/ppr/D2-Conrad/S4/Office_Conrad-S4.csv",index_col='timestamp')

In [4]:
len(df)

8640

In [5]:
train_data = df.iloc[:7920]
test_data = df.iloc[7920:]

In [6]:
len(train_data),len(test_data)

(7920, 720)

In [7]:
def load_data(data, seq_len):
    X = []
    y = []
    #print(len(data))
    for i in range(seq_len, len(data),12):
        X.append(data.iloc[i-seq_len : i, 0])
        y.append(data.iloc[i:i+12, 0])
        #print(y)
   
    
    return [X, y]

In [8]:
# Sequence of training data
seq_len = 24 #choose sequence length

X1, y1= load_data(train_data, seq_len)

In [9]:
#Sequence of testing data
seq_len = 24 #choose sequence length

X2, y2= load_data(test_data, seq_len)

In [10]:
X_train = X1      
y_train = y1
X_test = X2             
y_test = y2

In [11]:
# convert training and testing data to numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
  
X_test = np.array(X_test)
y_test = np.array(y_test)

# reshape data to input into 3 dimension
#X_train = np.reshape(X_train, (8016, seq_len, 1))
#X_test = np.reshape(X_test, (X_test.shape[0], seq_len, 1))

In [12]:
print('X_train.shape = ',X_train.shape)
print('y_train.shape = ',y_train.shape)
print('X_test.shape = ',X_test.shape)
print('y_test.shape = ',y_test.shape)

X_train.shape =  (658, 24)
y_train.shape =  (658, 12)
X_test.shape =  (58, 24)
y_test.shape =  (58, 12)


In [13]:
import tensorflow as tf
import sklearn.preprocessing
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model

In [14]:
input_layer = Input(shape=24, dtype='float32')
dense1 = Dense(24, activation='linear')(input_layer)
dense2 = Dense(18, activation='linear')(dense1)
dense3 = Dense(18, activation='linear')(dense2)
dropout_layer = Dropout(0.2)(dense2)
output_layer = Dense(12, activation='linear')(dropout_layer)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 dense (Dense)               (None, 24)                600       
                                                                 
 dense_1 (Dense)             (None, 18)                450       
                                                                 
 dropout (Dropout)           (None, 18)                0         
                                                                 
 dense_3 (Dense)             (None, 12)                228       
                                                                 
Total params: 1,278
Trainable params: 1,278
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x=X_train, y=y_train, batch_size=128, epochs=300)

Epoch 1/300
6/6 [==============================] - 1s 3ms/step - loss: 282.8008
Epoch 2/300
6/6 [==============================] - 0s 4ms/step - loss: 169.3327
Epoch 3/300
6/6 [==============================] - 0s 3ms/step - loss: 105.6405
Epoch 4/300
6/6 [==============================] - 0s 3ms/step - loss: 68.8372
Epoch 5/300
6/6 [==============================] - 0s 5ms/step - loss: 49.1321
Epoch 6/300
6/6 [==============================] - 0s 3ms/step - loss: 38.5217
Epoch 7/300
6/6 [==============================] - 0s 3ms/step - loss: 30.1089
Epoch 8/300
6/6 [==============================] - 0s 3ms/step - loss: 25.8699
Epoch 9/300
6/6 [==============================] - 0s 3ms/step - loss: 23.3393
Epoch 10/300
6/6 [==============================] - 0s 3ms/step - loss: 21.1662
Epoch 11/300
6/6 [==============================] - 0s 3ms/step - loss: 19.9036
Epoch 12/300
6/6 [==============================] - 0s 3ms/step - loss: 18.2510
Epoch 13/300
6/6 [============================

In [16]:
train_prediction=model.predict(X_train)
test_prediction=model.predict(X_test)

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt

In [18]:
rmse_train = sqrt(mean_squared_error(y_train, train_prediction))
rmse_test = sqrt(mean_squared_error(y_test, test_prediction))

mae_train=mean_absolute_error(y_train, train_prediction)
mae_test=mean_absolute_error(y_test, test_prediction)

mape_train = mean_absolute_percentage_error(y_train, train_prediction)
mape_test = mean_absolute_percentage_error(y_test, test_prediction)

In [19]:
print('Root Mean Squared Error for ANN Model for training is:',rmse_train)
print('Root Mean Squared Error for ANN Model for testing is:',rmse_test)
print('Mean Absolute Error for ANN Model for training is:',mae_train)
print('Mean Absolute Error for ANN Model for testing is:',mae_test)
print('MAPE for ANN Model for training is:',mape_train)
print('MAPE for ANN Model for testing is:',mape_test)

Root Mean Squared Error for ANN Model for training is: 1.447122638079734
Root Mean Squared Error for ANN Model for testing is: 1.5638716012104041
Mean Absolute Error for ANN Model for training is: 1.0660567018769582
Mean Absolute Error for ANN Model for testing is: 1.15050135535021
MAPE for ANN Model for training is: 0.11484105487389323
MAPE for ANN Model for testing is: 0.13549625405615243


In [ ]:
#pred=model.predict(X_test)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (24,8)
plt.plot(test_prediction,label='LSTM_Predictions')
plt.plot(y_test,label='Actual value')
plt.legend(loc="upper left")
plt.show()

In [ ]:
y_test=y_test.reshape(-1,1)

y_test[0],test_prediction[0]

(array([2.54999995]), array([2.2474155], dtype=float32))

In [ ]:
d1=[]
for i in range(len(y_test)):
    d1.append(y_test[i]-test_prediction[i])
    
print(d1)

[array([0.3025844], dtype=float32), array([0.50469756], dtype=float32), array([-0.09460044], dtype=float32), array([0.31566167], dtype=float32), array([-0.00765181], dtype=float32), array([1.0319862], dtype=float32), array([0.44412303], dtype=float32), array([0.19131255], dtype=float32), array([0.23618555], dtype=float32), array([0.29882288], dtype=float32), array([0.47502637], dtype=float32), array([-0.6382477], dtype=float32), array([-0.692958], dtype=float32), array([0.5071014], dtype=float32), array([-0.11342168], dtype=float32), array([0.5132935], dtype=float32), array([-0.25803447], dtype=float32), array([1.5235887], dtype=float32), array([-0.09268188], dtype=float32), array([0.05426645], dtype=float32), array([1.3116488], dtype=float32), array([0.8018553], dtype=float32), array([-0.59282804], dtype=float32), array([-0.5703342], dtype=float32), array([-0.0925734], dtype=float32), array([-0.5297289], dtype=float32), array([-0.01041937], dtype=float32), array([-0.3175304], dtype=fl

In [ ]:
d1.sort()
# Range of error value
max(d1),min(d1)

(array([2.471511], dtype=float32), array([-1.4796157], dtype=float32))

In [ ]:
neg=[]
pos=[]
for i in range(len(d1)):
  if(d1[i]<0):
    neg.append(d1[i])
  if(d1[i]>0):  
    pos.append(d1[i])

In [ ]:
len(neg),len(pos)

(346, 350)